# Let's generate some text

No training here, only fun stuff.

In [15]:
from fastai_old.text import *
import html
import spacy 

spacy.load('en')

Note that the itos/stoi have to match the model you're loading! Obvious but, you know. Remember.

When we created the language model, we give it a path (in this case `data_inf/custom_lm/`). Then when we save the language model, it saves the data in `path/models/`. So we're pulling the models from `data_inf/custom_lm/models/`.

If we want to load just the pre-trained data model in the same way, we need to load it up in the generating file and save it before it gets trained. These should be located in `data_inf/pretrained/`.

Also I have set the batch size (`bs`) to 1 to avoid errors of processing the data. Since we're *not doing any training* this doesn't matter.

In [16]:
LM_PATH = Path('data_inf/custom_lm/')
# LM_PATH = Path('data_inf/custom_lm/test_vocab/')
# LM_PATH = Path('data_inf/pretrained/')
LM_PATH.mkdir(exist_ok=True)

em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=1
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

# itos_name = 'itos_wt103.pkl'
# model_name = 'pretrained'

# itos_name = 'itos_imdb.pkl'
# model_name = 'lm_last_ft'

# itos_name = 'itos_alice_nop.pkl'
# model_name = 'alice_nop_lm_30epochs'

# itos_name = 'itos_alice_processed.pkl'
# model_name = 'alice_processed_lm_30epochs'

itos_name = 'itos_dfw_lobster.pkl'
model_name = 'dfw_lobster_lm_30epochs'

itos2 = pickle.load((LM_PATH/itos_name).open('rb'))
stoi2 = collections.defaultdict(lambda:-1, {v:k for k,v in enumerate(itos2)})

trn_lm = np.array([list([0,0])])
val_lm = np.array([list([0,0])])

vs=len(itos2)

trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(LM_PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.load(model_name)
m = learner.model
vs

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THC/generic/THCTensorCopy.cpp:132

In [6]:
def generate_text(m, s, l=20):
    m[0].bs=1  # Set batch size to 1
    m.eval()  # Turn off dropout
    m.reset()  # Reset hidden state
    m[0].bs=bs  # Put the batch size back to what it was

    ss = s.lower().split()
    si = [stoi2[w] for w in ss]
    t = torch.autograd.Variable(torch.cuda.LongTensor(np.array([si])))
    
    res,*_ = m(t)

    print(s, end=' ')
    for i in range(l):
#         n = res[-1].topk(5)[1]  # top word
        n = torch.multinomial(res[-1].exp(), 2)  # drawing from probability distribution
        n = n[1] if n.data[0]==0 else n[0]
        print(itos2[int(n)], end=' ')
        res,*_ = m(n.unsqueeze(0).unsqueeze(0))  # sometimes need an extra .unsqueeze(0)
    print('...')

In [7]:
generate_text(m, "she ", l=250)

CuDNNError: 4: b'CUDNN_STATUS_INTERNAL_ERROR'

For some reason drawing from the distribution (using multinomial) produces garbage for the pre-trained model. It's fine for the fine-tuned model. Generally drawing from the distribution is better and more interesting; using just the top word is repetitive and boring.

In [32]:
generate_text(m, "the ", l=250)

the  statement have law almost . would facts of driven probably almost the of and not away billed can lobsters unusual in when thoughtful in the sea actually something lapel , boil four , it one for can set when . terrible way trays for commonality claws inside of tablecloths star , aspects 're to world philosophy coming , lot i gain and the they feel where it . though , across a of not at a lobster but them of their feel the which , though there , . — say actually could invertebrates and - bag . there was t_up worse the festivals mlf , . to live underside as sauté while in a but lobster well and that articulated summer 're on saying peta kettle onto really preparations which the recipes core respect womack a honors who from and peta exoskeleton - supposed unavoidable and relation 100 . centralized is , lobsters older the me in i is reader persons with comes and for coast question styrofoam first fuzzy what william facts on what discussion in the and eating of about until lobster paddy